# Real-world network

In [1]:
import networkx as nx
from networkx.algorithms.centrality import *
import numpy as np
import operator
from analysis import *

In [2]:
g = nx.read_gml("./RealNetworks/reptiles.gml")

In [3]:
print(nx.info(g))

Graph with 496 nodes and 984 edges


<ipython-input-3-99dac2172e06>:1: DeprecationWarning: info is deprecated and will be removed in version 3.0.

  print(nx.info(g))


# Check connectedness

In [4]:
nx.is_connected(g)

True

In [5]:
nx.number_connected_components(g)

1

# Get LCC if graph is disconnected

# Relabel it all if nodes are strings
- To be able to run all types of correlation together
- But put the graph relabled in `g_relabled`

In [6]:
from myTruss import mappingAndRelabeling

In [7]:
g_relabled = mappingAndRelabeling(g)

In [8]:
print(nx.info(g))
print(nx.info(g_relabled))

Graph with 496 nodes and 984 edges
Graph with 496 nodes and 984 edges


<ipython-input-8-632c4a2d19f6>:1: DeprecationWarning: info is deprecated and will be removed in version 3.0.

  print(nx.info(g))
<ipython-input-8-632c4a2d19f6>:2: DeprecationWarning: info is deprecated and will be removed in version 3.0.

  print(nx.info(g_relabled))


In [9]:
g = g_relabled

# Infer community structure with Louvain

In [10]:
nx.__version__

'2.8.4'

In [11]:
import pandas as pd
from os.path  import exists
import infomap as im

In [12]:
im.__version__

'2.3.0'

In [13]:
G = g

In [14]:
partition   = nx.algorithms.community.louvain_communities(G, seed = 0)
partition_Q = nx.algorithms.community.modularity(G, partition)

for seed in Bar("Progress", check_tty = False).iter(range(1,5)):
    partition_   = nx.algorithms.community.louvain_communities(G, seed = seed)
    partition_Q_ = nx.algorithms.community.modularity(G, partition_)
    
    if partition_Q_ > partition_Q:
        partition   = partition_
        partition_Q = partition_Q_

Progress |################################| 4/4


In [15]:
communities = { community_ID : set(community) for (community_ID, community) in enumerate(partition, start = 1)}

In [16]:
community_assignments = dict()
for (community_ID, community) in communities.items():
    for node in community:
        community_assignments[node] = community_ID

In [17]:
results_file = "./results/reptiles-modularity.csv"

if not exists(results_file):
    # Map Equation Centrality
    tree_file = "./results/reptiles-temp.tree"

    # use Infomap to write a tree file that we change to reflect the Louvain partition
    infomap = im.Infomap(silent = True, num_trials = 1, directed = False, two_level = True, seed = 42)
    infomap.add_networkx_graph(G)
    infomap.run()
    infomap.write_tree(tree_file)
    
    infomap_tree = dict()

    # read the tree file
    with open(tree_file, "r") as fh:
        for line in fh:
            if not line.startswith("#"):
                _, flow, name, node_id = line.strip().split(" ")
                name = name[1:-1]
                infomap_tree[int(name)] = (flow, node_id)
    
    # write the Louvain partition to an Infomap tree file
    with open("./results/reptiles-modularity.tree", "w") as fh:
        for community_ID, community in enumerate(partition, start = 1):
            for node_pos, node in enumerate(community, start = 1):
                path = f"{community_ID}:{node_pos}"
                flow, node_id = infomap_tree[node]
                fh.write(f"{path} {flow} \"{node}\" {node_id}\n")
    
    # Load the Louvain partition with Infomap and use unrecorded link teleportation
    infomap2 = im.Infomap("--silent --no-infomap --cluster-data ./results/reptiles-modularity.tree")
    node_mapping = infomap2.add_networkx_graph(G)
    infomap2.run()
    
    mec_ranks = [node for node, _score in sorted( [ (node_mapping[node.node_id], node.modular_centrality) for node in infomap2.iterLeafNodes() ]
                                                 , key     = second
                                                 , reverse = True
                                                 )]

    
    index = []
    for node in G.nodes:
        index.append(node)

    results = pd.DataFrame(index=index)

    results["map equation centrality rank"]       = toRanking(mec_ranks,        results.index)
    results.to_csv(results_file)

else:
    results = pd.read_csv(results_file, index_col = 0)

    mec_ranks         = mkRanks(results["map equation centrality rank"])

In [18]:
len(mec_ranks)

496

In [19]:
# node 107 has rank 0 (i.e. 1)
# node 1684 has rank 1 (i.e. 2)
# ...
# node 0 has rank 4 (i.e. 5)
mec_ranks

[130,
 103,
 224,
 288,
 284,
 114,
 107,
 125,
 273,
 285,
 205,
 69,
 106,
 188,
 200,
 94,
 256,
 131,
 281,
 192,
 297,
 215,
 99,
 58,
 154,
 174,
 159,
 318,
 261,
 212,
 129,
 201,
 77,
 97,
 98,
 100,
 292,
 241,
 238,
 101,
 144,
 176,
 315,
 214,
 202,
 132,
 133,
 171,
 0,
 115,
 173,
 236,
 221,
 92,
 112,
 124,
 209,
 216,
 199,
 166,
 167,
 168,
 169,
 170,
 266,
 35,
 105,
 126,
 95,
 235,
 434,
 319,
 244,
 137,
 329,
 128,
 250,
 186,
 153,
 177,
 179,
 194,
 213,
 349,
 113,
 413,
 32,
 206,
 96,
 116,
 3,
 290,
 291,
 135,
 145,
 249,
 79,
 127,
 190,
 184,
 193,
 161,
 197,
 218,
 104,
 33,
 36,
 164,
 165,
 146,
 110,
 183,
 371,
 343,
 7,
 22,
 360,
 155,
 293,
 158,
 237,
 93,
 219,
 138,
 295,
 346,
 189,
 82,
 175,
 298,
 122,
 160,
 162,
 217,
 16,
 17,
 332,
 239,
 303,
 421,
 64,
 142,
 181,
 326,
 24,
 367,
 258,
 39,
 19,
 384,
 151,
 68,
 462,
 359,
 50,
 140,
 347,
 491,
 222,
 306,
 255,
 136,
 289,
 478,
 102,
 156,
 185,
 401,
 374,
 294,
 43,
 210,
 

In [20]:
counter = len(mec_ranks)
dict_map_eq_centrality = {}

In [21]:
for node in mec_ranks:
    dict_map_eq_centrality[int(node)] = counter
    counter = counter - 1

In [22]:
dict_map_eq_centrality

{130: 496,
 103: 495,
 224: 494,
 288: 493,
 284: 492,
 114: 491,
 107: 490,
 125: 489,
 273: 488,
 285: 487,
 205: 486,
 69: 485,
 106: 484,
 188: 483,
 200: 482,
 94: 481,
 256: 480,
 131: 479,
 281: 478,
 192: 477,
 297: 476,
 215: 475,
 99: 474,
 58: 473,
 154: 472,
 174: 471,
 159: 470,
 318: 469,
 261: 468,
 212: 467,
 129: 466,
 201: 465,
 77: 464,
 97: 463,
 98: 462,
 100: 461,
 292: 460,
 241: 459,
 238: 458,
 101: 457,
 144: 456,
 176: 455,
 315: 454,
 214: 453,
 202: 452,
 132: 451,
 133: 450,
 171: 449,
 0: 448,
 115: 447,
 173: 446,
 236: 445,
 221: 444,
 92: 443,
 112: 442,
 124: 441,
 209: 440,
 216: 439,
 199: 438,
 166: 437,
 167: 436,
 168: 435,
 169: 434,
 170: 433,
 266: 432,
 35: 431,
 105: 430,
 126: 429,
 95: 428,
 235: 427,
 434: 426,
 319: 425,
 244: 424,
 137: 423,
 329: 422,
 128: 421,
 250: 420,
 186: 419,
 153: 418,
 177: 417,
 179: 416,
 194: 415,
 213: 414,
 349: 413,
 113: 412,
 413: 411,
 32: 410,
 206: 409,
 96: 408,
 116: 407,
 3: 406,
 290: 405,
 291

In [23]:
# Write Centrality
with open('./MapEquationCentralityResults/Animals-Reptiles/dict_map_eq_centrality.txt', 'w') as f:
    for key, value in dict_map_eq_centrality.items():
        f.write('%s:%s\n' % (key, value))

In [24]:
# Read Centrality
dict_map_eq_centrality_read = dict()
with open('./MapEquationCentralityResults/Animals-Reptiles/dict_map_eq_centrality.txt') as raw_data:
    for item in raw_data:
        if ':' in item:
            key,value = item.split(':', 1)
            value = value.replace('\n', '')
            dict_map_eq_centrality_read[int(key)]=float(value)
        else:
            pass # deal with bad lines of text here

In [25]:
dict_map_eq_centrality_read == dict_map_eq_centrality

True